## Importing Libraries and Data

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Importing Data
path = r'C:\Users\javir\Downloads\Javis Data Analytics stuff\Achievement 4\Instacart Basket Analysis'
df = pd.read_pickle(os.path.join(path,'02 Data', 'Prepared Data', 'orders_products_merged_added.pkl'))

## Question 2 - Aggregate mean of order_number by department_id for entire dataframe

In [3]:
# Checking the scope of our dataframe
df.shape
# 32,641,268 rows

(32641268, 20)

In [4]:
# Grouping our dataframe by department_id, then grabing the mean of order_number and grouping it by department_id
df.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1.0,15.457838
2.0,17.277920
3.0,17.170395
4.0,17.811403
5.0,15.215751
6.0,16.439806
7.0,17.225802
8.0,15.340650


## Question 3 - Analyze the results

##### While the numbers did change, there was no drastic leaps frim our 1 million to 32 million row change. If we took only the whole number, departments: 1,8,14,15,16,20 all had a change of only 1 numeral point. Departments: 1 and 15 went up 1 average point, and Departments: 8,14,16,20 all went down 1 average point. Having 32x's the amount of data while a huge benefactor, of average orders we did not see any large changes.

## Question 4 - Create loyalty flags for the entire dataframe

In [5]:
# Creating a new column with max_order numbers to use new data for creating our layalty flags
df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

### Creating our loyalty flag columns and filling in the data

In [6]:
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [7]:
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular Customer'

In [8]:
df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

## Question 5 - Marketing team wants to know spending habits of the three groups

In [9]:
# Pulling statistical information based on our customer groups we defined
df.groupby('loyalty_flag').agg({'prices':['mean','min','max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.377144  1.0  99999.0
New customer      13.208437  1.0  99999.0
Regular Customer  12.616225  1.0  99999.0

### Spending habits among the three groups are similar, however, Newer customers spend more on items while Loyal customers spend the least. Max value seems to have an error as all three categories show a purchase of 99,999

## Question 6 -  Create a spending flag based on average price spent on orders

In [10]:
# Creating a new column with avg_spent to be used for creating our spending_flags
df['avg_spent'] = df.groupby(['user_id'])['prices'].transform(np.mean)

In [11]:
# Checking to ensure that our transform() created an average_spent across all orders, and only one value was inputed
# Since all 3 categories of statistical info is the same, the function was successful
df.groupby('user_id').agg({'avg_spent':['mean','min','max']})

avg_spent                    
             mean       min       max
user_id                              
1        6.367797  6.367797  6.367797
2        7.515897  7.515897  7.515897
3        8.197727  8.197727  8.197727
4        8.205556  8.205556  8.205556
5        9.189189  9.189189  9.189189
...           ...       ...       ...
206205   8.909375  8.909375  8.909375
206206   7.646667  7.646667  7.646667
206207   7.313453  7.313453  7.313453
206208   8.366617  8.366617  8.366617
206209   7.058915  7.058915  7.058915

[206209 rows x 3 columns]

##### Creating our spender_flags

In [12]:
df.loc[df['avg_spent'] < 10, 'spender_flag'] = 'Low spender'

In [13]:
df.loc[df['avg_spent'] >= 10, 'spender_flag'] = 'High spender'

In [14]:
# Checking flag_counts to ensure they match our rows
df['spender_flag'].value_counts()

Low spender     32001268
High spender      640000
Name: spender_flag, dtype: int64

## Question 7 - Creating frequency_flags to send notifications to customers who haven't ordered recently

In [15]:
# Creating our new column
df['order_frequency'] = df.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

##### Creaing our frequency_flags

In [16]:
df.loc[df['order_frequency'] < 20, 'frequency_flag'] = 'Non-frequent customer'

In [17]:
df.loc[(df['order_frequency'] > 10) & (df['order_frequency'] <= 20), 'frequency_flag'] = 'Regular customer'

In [18]:
df.loc[df['order_frequency'] >= 10, 'frequency_flag'] = 'Frequent customer'

In [19]:
# Checking flag_counts to ensure they match our rows
df['frequency_flag'].value_counts()

Non-frequent customer    20355757
Frequent customer        12285511
Name: frequency_flag, dtype: int64

## Question 8 - Export

In [21]:
df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merg_flag.pkl'))